In [1]:
# Import the dependencies.  Challenge
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import json 
import requests
import timeit

from datetime import datetime
from config import weather_api_key

In [2]:
# Create a set of 2,000 random latitude and longitude combinations. 
lats = np.random.uniform(low=-90.000, high=90.000, size=2100)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2100)

# Unpack lat_lngs zip object into a list. Note only can unzip a zipped tuple once before removed from pc mem
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Use the print() function to display the latitude and longitude combinations in list. Will create ordered pairs of lats & longs
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

-50.21107604107446 129.16627475538502
35.569899342688245 109.9298496383434
-71.65931479866614 -168.5107122724437
-38.210607434678735 -72.59734078133042
74.13724490499152 -111.72262252930085
-69.96567229134325 73.89654880083009
-4.373560605557401 -46.75940853861232
44.23010589245641 -31.77141794854245
63.89710706180594 101.33763417013051
23.939127907733976 -166.84124617461683
-57.396329956977276 36.74858076376708
66.71422951214024 55.183753536799884
-41.93614718727878 139.32341165833066
-23.3804470588029 16.811669777669636
-27.098455890373238 48.23757818998564
16.468136780200112 75.07688530579182
-32.628193375624214 83.00538233236944
17.171405912525856 62.260301806598164
-39.35163838735596 6.189914649821873
17.782669562358166 -76.63652545671044
-22.90966637754856 -136.19746579219697
1.5663662948627461 141.7201165442363
-8.939863375849058 -16.73024732834685
-55.21600347667595 -157.9316091716185
63.424152937164536 -142.06030246333626
-64.36427734319275 77.19371819255906
21.463084684774458

19.3725421760776 -86.11978481140629
-65.18879168124114 -130.38466022521263
63.09227581399378 140.99466927174961
24.773868553809024 31.518446483504647
20.18688913722528 90.0348919092661
-8.002609433143618 112.50011872643341
56.701671911476126 45.330366443870844
-66.18401174846076 157.028733416386
3.2056457506149485 28.79948000544522
-64.52133678634938 44.21085695066813
86.79537476791424 -4.869148246778337
-44.6848061719872 54.081481365224136
19.911718911784007 102.86957192779448
34.81655382127869 -162.28729982009654
29.597272200681942 -171.87523684637952
16.065241519316714 16.35081425653547
39.69570121417914 149.06110070898382
-67.18056025071004 -32.06292509964152
-5.242001088690188 -67.3115297086097
69.8270854071304 105.70469720778152
51.23094958966476 -151.76431701103948
55.64760080830197 -128.88573545032756
-59.998954649284876 160.53364889564028
-3.8588218077603784 -2.058484741914185
-77.42386247659556 48.001208061425245
46.21814043561591 145.35023573468874
61.68589986285599 -167.839

46.67216339194269 106.30583073170607
61.12900107506593 -21.03664454643308
65.43627454612738 51.17445997632447
-49.21786358970989 -169.44011695580545
-49.86116166781453 152.61820161160045
78.965613568457 83.07207200027733
56.190383230314296 178.57824646432567
-22.68886222300874 -141.86868700617669
1.0863345892725675 -114.4590306489525
15.951447356121093 126.30209950180534
80.79435146238407 56.00207190117399
52.78045801713674 -9.164532344015868
-41.28816810319039 -147.87167301432913
-51.2255620918466 2.062060844052752
32.01021004244738 -152.9689986183739
43.52576976506367 106.41328138633673
45.33410861329435 -136.5717776464611
-70.83259794843156 49.87772908263119
1.6293507837072667 -23.828251973635815
0.9219441694873467 -156.88633158277165
-54.918767451401216 113.09609248845544
-80.88634727164899 -71.87767537141194
-32.65436971805021 -7.035607769543645
-42.39843135126039 -30.357933368408368
-59.49877115531642 -107.98084604759886
-69.59678181831481 -77.7176720544436
-88.1517697484555 -51.

In [5]:
# Get the nearest city using the citipy module
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

797

In [6]:
# Perform an API call with the OpenWeatherMap
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [7]:
# Retrieve the following information from the API call:
# Latitude and longitude
# Maximum temperature
# Percent humidity
# Percent cloudiness
# Wind speed
# Weather description (for example, clouds, fog, light rain, clear sky)

# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

     # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_country = city_weather["sys"]["country"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        description = city_weather['weather'][0]['description']
        
        # Convert the date to ISO
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information to the city_data list
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description":description })
    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")  

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | port lincoln
Processing Record 2 of Set 1 | yanan
City not found. Skipping...
Processing Record 3 of Set 1 | vaini
Processing Record 4 of Set 1 | traiguen
Processing Record 5 of Set 1 | yellowknife
Processing Record 6 of Set 1 | saint-philippe
Processing Record 7 of Set 1 | amarante do maranhao
Processing Record 8 of Set 1 | lagoa
Processing Record 9 of Set 1 | tura
Processing Record 10 of Set 1 | kapaa
Processing Record 11 of Set 1 | port alfred
Processing Record 12 of Set 1 | izhma
Processing Record 13 of Set 1 | portland
Processing Record 14 of Set 1 | rehoboth
Processing Record 15 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 16 of Set 1 | rabkavi
Processing Record 17 of Set 1 | bambous virieux
Processing Record 18 of Set 1 | sur
Processing Record 19 of Set 1 | saldanha
Processing Record 20 of Set 1 | morant bay
Processing Record 21 of Set 1 | rikitea
Processing Record 22

Processing Record 41 of Set 4 | deputatskiy
Processing Record 42 of Set 4 | qasigiannguit
Processing Record 43 of Set 4 | hilo
Processing Record 44 of Set 4 | aleksandrov gay
Processing Record 45 of Set 4 | talcahuano
Processing Record 46 of Set 4 | juneau
Processing Record 47 of Set 4 | nanortalik
Processing Record 48 of Set 4 | geraldton
Processing Record 49 of Set 4 | moa
Processing Record 50 of Set 4 | tambopata
Processing Record 1 of Set 5 | bajil
Processing Record 2 of Set 5 | chimbote
Processing Record 3 of Set 5 | svetlogorsk
Processing Record 4 of Set 5 | kiunga
Processing Record 5 of Set 5 | tosya
Processing Record 6 of Set 5 | solovetskiy
City not found. Skipping...
Processing Record 7 of Set 5 | muyezerskiy
Processing Record 8 of Set 5 | chokurdakh
Processing Record 9 of Set 5 | carnarvon
Processing Record 10 of Set 5 | tsumeb
Processing Record 11 of Set 5 | rio esteban
Processing Record 12 of Set 5 | samdrup jongkhar
Processing Record 13 of Set 5 | arawa
Processing Record 

Processing Record 32 of Set 8 | kloulklubed
Processing Record 33 of Set 8 | tigre
Processing Record 34 of Set 8 | brigg
Processing Record 35 of Set 8 | namatanai
Processing Record 36 of Set 8 | kinablangan
Processing Record 37 of Set 8 | macas
Processing Record 38 of Set 8 | saleaula
City not found. Skipping...
Processing Record 39 of Set 8 | marawi
Processing Record 40 of Set 8 | grand river south east
City not found. Skipping...
Processing Record 41 of Set 8 | verkhnevilyuysk
Processing Record 42 of Set 8 | scottsburgh
City not found. Skipping...
Processing Record 43 of Set 8 | rio gallegos
Processing Record 44 of Set 8 | iqaluit
Processing Record 45 of Set 8 | tautira
Processing Record 46 of Set 8 | ambovombe
Processing Record 47 of Set 8 | nadym
Processing Record 48 of Set 8 | faanui
Processing Record 49 of Set 8 | damavand
Processing Record 50 of Set 8 | slave lake
Processing Record 1 of Set 9 | tevriz
Processing Record 2 of Set 9 | port said
Processing Record 3 of Set 9 | leh
Pro

Processing Record 19 of Set 12 | muyinga
Processing Record 20 of Set 12 | campbellsville
Processing Record 21 of Set 12 | ternovka
Processing Record 22 of Set 12 | birjand
Processing Record 23 of Set 12 | santa luzia
Processing Record 24 of Set 12 | pangnirtung
Processing Record 25 of Set 12 | la ronge
Processing Record 26 of Set 12 | katherine
Processing Record 27 of Set 12 | rognan
Processing Record 28 of Set 12 | port macquarie
Processing Record 29 of Set 12 | fayaoue
Processing Record 30 of Set 12 | vadso
Processing Record 31 of Set 12 | mys shmidta
City not found. Skipping...
Processing Record 32 of Set 12 | dinanagar
Processing Record 33 of Set 12 | itoman
Processing Record 34 of Set 12 | price
Processing Record 35 of Set 12 | ziguinchor
Processing Record 36 of Set 12 | brownsville
Processing Record 37 of Set 12 | salaga
Processing Record 38 of Set 12 | armacao dos buzios
City not found. Skipping...
Processing Record 39 of Set 12 | plettenberg bay
Processing Record 40 of Set 12 |

Processing Record 49 of Set 15 | simao
Processing Record 50 of Set 15 | pamekasan
Processing Record 1 of Set 16 | kasongo-lunda
Processing Record 2 of Set 16 | boa vista
Processing Record 3 of Set 16 | rocha
Processing Record 4 of Set 16 | birin
Processing Record 5 of Set 16 | port-de-paix
City not found. Skipping...
Processing Record 6 of Set 16 | kupang
Processing Record 7 of Set 16 | miri
Processing Record 8 of Set 16 | peterhead
Processing Record 9 of Set 16 | san anselmo
Processing Record 10 of Set 16 | hovd
Processing Record 11 of Set 16 | asyut
Processing Record 12 of Set 16 | sorvag
City not found. Skipping...
Processing Record 13 of Set 16 | kapit
Processing Record 14 of Set 16 | suslovo
Processing Record 15 of Set 16 | gemena
Processing Record 16 of Set 16 | vila do maio
Processing Record 17 of Set 16 | skovorodino
Processing Record 18 of Set 16 | zhumadian
Processing Record 19 of Set 16 | mandalgovi
Processing Record 20 of Set 16 | pandan
Processing Record 21 of Set 16 | enn

In [8]:
# The number of cities in the city_data array of dictionaries
len(city_data)

732

In [9]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Port Lincoln,AU,-34.7333,135.8667,56.16,74,48,15.52,scattered clouds
1,Vaini,TO,-21.2000,-175.2000,75.36,83,75,10.36,broken clouds
2,Traiguen,CL,-38.2500,-72.6833,38.37,98,100,3.83,overcast clouds
3,Yellowknife,CA,62.4560,-114.3525,61.32,82,75,4.61,broken clouds
4,Saint-Philippe,RE,-21.3585,55.7679,74.79,69,28,15.17,scattered clouds
5,Amarante Do Maranhao,BR,-5.5667,-46.7422,70.36,74,43,3.83,scattered clouds
6,Lagoa,PT,39.0500,-27.9833,65.70,79,100,8.48,overcast clouds
7,Tura,IN,25.5198,90.2201,88.79,63,82,3.80,broken clouds
8,Kapaa,US,22.0752,-159.3190,80.58,77,40,16.11,scattered clouds
9,Port Alfred,ZA,-33.5906,26.8910,54.18,82,9,5.99,clear sky


In [12]:
# Export the DataFrame as a CSV file, and save it as WeatherPy_Database.csv in the Weather_Database folder
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")